># <b>Desafio: análise de comentários negativos dos consumidores com TF-IDF</b>  
> Instituição: PUCPR  
> Curso: Tecnologia em Inteligência Artifical Aplicada  
> Disciplina: Processamento de Linguagem Natural  
> Professor: Lucas Oliveira  
> Estudante: Carla Edila Silveira  
> Data: 20/04/2024  


In [1]:
# Importa bibliotecas
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import pandas as pd
import numpy as np
import scipy.sparse as sp
from numpy.linalg import norm

In [2]:
# Define o corpus
CORPUS = [
'No dia 16/02 recebo um e-mail da Loja alegando divergência de dados e acusando pagamento não autorizado! Sem nenhum detalhe adicional! Simplesmente negam a compra, a fatura permanece, e não explicam nada!Liguei para o atendimento e tive a mesma informação. Fui bem atendido. Como trabalho muito com notebook, realizei novamente a compra e fui informado que o pagamento é estornado em até 48h. Quando entrei no Reclame Aqui, que arrependimento tive! Reclamações de pessoas que há mais de uma semana não recebem o estorno, e de problemas similares ao meu! Espero sinceramente que o valor seja estornado logo e que a segunda compra seja bem procedida, e o produto entregue em tempo. Não tenho nenhum apreço por ter que recorrer a medidas legais! Espero profissionalismo e transparência da empresa, como obrigação com o consumidor!',
'Boa tarde, fiz um pedido de dois pneus dia 16/01/2020, quase na mesma hora recebi um e-mail de confirmação do pagamento, que a mercadoria já estava com a transportadora e que a previsão de retirada seria em até 7 dias úteis, achei ótimo, muito rápido, já que seria para colocar no carro para viajar no começo do mês, então fiquei muito satisfeito, ia dar tudo certo! Mais não foi isso q aconteceu, não recebi o e-mail de confirmação até a presente data, ai começou a dor de cabeça, mandei e-mail reclamando da demora do dito cujo e-mail de aprovação para retirada do produto no dia 31-01-2020 sendo q o prazo já tinha passado, a resposta foi a seguinte...Identificamos um problema operacional que atrasou a entrega do seu pedido 281425025, mas já estamos em contato com a transportadora para resolvermos o mais rápido possível.Não se preocupe, acionamos nossa equipe de suporte pelo protocolo 6803089. Já estamos acompanhando todo o processo e retornaremos o contato em 05 (cinco) dias úteis.Pedimos desculpas pelo transtorno e estamos à sua disposição.Protocolo: 6779662 Atenciosamente,Atendimento.com. Pois é a o prazo passou e ninguém entrou em contato, foi aí que mandei outro e-mail, e a resposta já foi outra de três dias.. Já acionamos a área responsável. Não se preocupe, em 03 (três) dias corridos o setor responsável entrará em contato.Pois bem ninguém entrou em contato, mas eu entrei em contato por telefone e quem me atendeu, falou em mais 5 dias para o contato comigo! Pow tá de brincadeira né , até quando isso?! Pior de tudo, é que eles tem o produto na loja, antes de comprar no site, passei na loja, mas estava bem mais caro, então optei por comprar no site e retirar na loja! Porém só libera a retirada do produto, com o e-mail de confirmação do site. Estou indignado com isso, pois já estou pagando o pneu no cartão, sendo que não tenho o produto em mãos.',
'Realizei a compra de 4 pneus no dia 21 de janeiro, e escolhi a opção de retirar na loja física, o prazo era de 7 dias. Até hoje não recebi a mercadoria, já liguei incontáveis vezes para o numero do atendimento e não obtenho nenhum tipo de retorno, no começo me falavam que eu tinha que aguardar o e-mail de confirmação para retirada da mercadoria na loja, e agora, quase um mês depois, a única coisa que me falam é que irão verificar e me retornar, mas não me retornam. No dia 05 de fevereiro recebi do atendimento@atendimento.com.br um e-mail (ANEXO) falando que iriam analisar o que aconteceu com o meu pedido e entrariam em contato dentro de 5 dias úteis, pois bem, estou até hoje aguardando e nada. Ontem fui na loja física e me informaram que não podem fazer nada, e que estão aguardando a transportadora. Saindo de lá liguei novamente no telefone do atendimento e fui informada pela atendente que a mercadoria havia voltado para o centro de distribuição pois não havia sido retirada. Em momento algum eu recebi e-mail que a mercadoria estava disponível para retirada, muito pelo contrário, todas as vezes que liguei me falaram que eu tinha que aguardar o e-mail. Inclusive quando fui na loja física a atendente não me informou que a mercadoria já tinha ido pra loja e retornado para o centro de distribuição, ela me disse que estava aguardando a transportadora e que não podia fazer absolutamente nada. É UM TOTAL DESCASO E FALTA DE RESPEITO COM O CONSUMIDOR, FICAMOS SEM RETORNO E AINDA RECEBEMOS INFORMAÇÕES DESENCONTRADAS, NINGUÉM RESOLVE O PROBLEMA, FALAM QUE IRÃO VERIFICAR E RETORNAR MAS NÃO RETORNAM. É A PRIMEIRA E SERÁ A ÚLTIMA VEZ QUE COMPRO NO SITE da Loja, VOCÊS SÃO PÉSSIMOS. ATENDIMENTO HORRÍVEL. COMPREI COM VOCÊS POR CONTA DO PRAZO QUE ESTAVA NO SITE DE VOCÊS POIS TINHA VIAGEM MARCADA, E OLHA SÓ, NÃO CONSEGUI VIAJAR POIS OS PNEUS NÃO CHEGARAM A TEMPO. COMPRAR COM VOCÊS É SÓ PREJUÍZO, NÃO RECOMENDO PARA NINGUÉM.',
'No dia 11/02/2020 realizei um pedido de compras on-line no site da Loja. Na tela principal aparece uma promoção do creme dental siga sorrindo, no banner está escrito: compre 3 produtos do creme dental e concorra a R$1000,00* + FRETE GRÁTIS NAS COMPRAS ACIMA DE 39,90. Clicando no banner não existe nenhum regulamento nem mais informações, desta forma inclui na minha compra 3 produtos do creme dental no carrinho e no mesmo momento o valor do frete passou para 0 reais. Conclui a compra no valor de R$391,78 e quando foi cobrado no cartão haviam incluído o frete de R$14,90. As compras chegaram certas e resolvi fazer outro pedido (n 308435373). Ainda constava o mesmo banner de promoção então inclui os 3 produtos do creme dental. Novamente ocorreu o mesmo erro e o frete foi cobrado. Liguei no SAC (protocolo 6918522) e o atendente  me falou que na verdade ao comprar os 3 produtos do creme dental estaria concorrendo aos R$1000,00 da promoção e também concorrendo a um frete grátis. Argumentei que isso era uma propaganda enganosa, pois não estava claro a nenhum momento e não teria lógica participar de um sorteio para ganhar frete grátis!?!Recebi estas segundas compras certas, porém não obedeceram o horário escolhido no site e entregaram 1h antes do combinado, por sorte tinha alguém para receber naquele momento.Entrando no site da promoção do creme dental, existe sim a inclusão dos 3 produtos para concorrer aos R$1000,00 porém não existe nada sobre frete grátis muito menos vinculado aoà Loja. Desta forma solicito a devolução do valor dos fretes.'
]
new_doc = ['fiz uma compra dia 17/01/2020 e solicitei o máximo de parcelas no caso 15x, ai fui conferir e vi que foi parcelado apenas em 8x, liguei logo depois de 2h pra verificar e solicitar que fosse alterado para 15x assim como eu tinha solicitada, me comunicaram que eu teria que cancelar e fazer novamente a compra, até ai tudo bem eu solicitei o cancelamento com eles para comprar novamente, porém já se passaram 27 dias apás a solicitação do estorno do cartão de crédito, já efetuei várias ligações com dois protocolos em mãos e nada de ser feito o estorno, tive que pagar a primeira parcela que foi lançada na fatura e nada deles estornarem meu cartão, a segunda parcela já esta por vir e nada de estornarem a compra ou liberarem o limite do meu cartão.']

In [3]:
# Define função bagOfWords
def bagOfWords(corpus, ngram_range=(1,1)):
    vectorizer = CountVectorizer(min_df=1, ngram_range=ngram_range)
    caracteristicas = vectorizer.fit_transform(corpus)
    return vectorizer, caracteristicas

In [4]:
# Aplica a função no corpus
bow_vectorizer, bow_caracteristicas = bagOfWords(CORPUS)
caracteristicas = bow_caracteristicas.todense()

print (caracteristicas)

[[0 0 1 ... 0 0 0]
 [0 2 0 ... 0 0 2]
 [0 0 0 ... 1 1 1]
 [3 0 1 ... 0 0 0]]


In [5]:
# Aplica a função em new_doc
new_doc_caracteristicas = bow_vectorizer.transform(new_doc)
new_doc_caracteristicas = new_doc_caracteristicas.todense()
print (new_doc_caracteristicas)

[[0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 3 0
  0 0 0 0 0 0 0 0 0 2 0 0 0 0 1 3 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 5 0 0 0 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 3 1 0 0 0 1 2 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 3 0 0 0 0 0 0 0 0 1 1 0 0 0 1 2
  0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0
  1 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 2 0 0 0 1 0 1 3 0 0 0 0 0 0 1 0 0 2 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1
  0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 6 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 1 0 0 1 0
  0 1 0 0 0 0 0 0 0 0 0 0 0 0]]


In [6]:
# Retorna lista com nomes das features (palavras únicas) que...
# foram extraídas dos documentos durante a vetorização do texto.
feature_names = bow_vectorizer.get_feature_names_out()
print (feature_names)

['00' '01' '02' '03' '05' '1000' '11' '14' '16' '1h' '2020' '21'
 '281425025' '308435373' '31' '39' '391' '48h' '6779662' '6803089'
 '6918522' '78' '90' 'absolutamente' 'achei' 'acima' 'acionamos'
 'acompanhando' 'aconteceu' 'acusando' 'adicional' 'agora' 'aguardando'
 'aguardar' 'ai' 'ainda' 'alegando' 'algum' 'alguém' 'analisar' 'anexo'
 'antes' 'ao' 'aos' 'aoà' 'aparece' 'apreço' 'aprovação' 'aqui'
 'argumentei' 'arrependimento' 'as' 'atenciosamente' 'atendente' 'atendeu'
 'atendido' 'atendimento' 'atrasou' 'até' 'autorizado' 'aí' 'banner' 'bem'
 'boa' 'br' 'brincadeira' 'cabeça' 'caro' 'carrinho' 'carro' 'cartão'
 'centro' 'certas' 'certo' 'chegaram' 'cinco' 'claro' 'clicando' 'cobrado'
 'coisa' 'colocar' 'com' 'combinado' 'começo' 'começou' 'comigo' 'como'
 'compra' 'comprar' 'compras' 'compre' 'comprei' 'compro' 'conclui'
 'concorra' 'concorrendo' 'concorrer' 'confirmação' 'consegui' 'constava'
 'consumidor' 'conta' 'contato' 'contrário' 'corridos' 'creme' 'cujo' 'da'
 'dados' 'd

In [7]:
#Para apresentar as caracteristicas do texto em forma de dataframe
def display_caracteristicas(caracteristicas, feature_names):
    df = pd.DataFrame(data=caracteristicas,
                      columns=feature_names)
    print (df)
    df.to_csv("saida.csv", encoding='utf-8')

display_caracteristicas(caracteristicas, feature_names)

   00  01  02  03  05  1000  11  14  16  1h  ...  viagem  viajar  vinculado  \
0   0   0   1   0   0     0   0   0   1   0  ...       0       0          0   
1   0   2   0   1   1     0   0   0   1   0  ...       0       1          0   
2   0   0   0   0   1     0   0   0   0   0  ...       1       1          0   
3   3   0   1   0   0     3   1   1   0   1  ...       0       0          1   

   vocês  voltado  área  ótimo  última  única  úteis  
0      0        0     0      0       0      0      0  
1      0        0     1      1       0      0      2  
2      4        1     0      0       1      1      1  
3      0        0     0      0       0      0      0  

[4 rows x 410 columns]


In [8]:
# Define a função tfidf
def tfidf_transformer(bow_matrix):
    transformer = TfidfTransformer(norm='l2', smooth_idf=True, use_idf=True)
    tfidf_matrix = transformer.fit_transform(bow_matrix)
    return transformer, tfidf_matrix

In [12]:
# Converte a matriz numpy new_doc_caracteristicas em array
# numpy antes de passá-lo para a função transform
new_doc_array = np.asarray(new_doc_caracteristicas)

# Obtém nomes das características
feature_names = bow_vectorizer.get_feature_names_out()

# Constroi transformador tfidf e mostra o corpus das características tfidf
tfidf_trans, tdidf_caracteristicas = tfidf_transformer(bow_caracteristicas)
caracteristicas = np.round(tdidf_caracteristicas.todense(), 2)
display_caracteristicas(caracteristicas, feature_names)

# Transforma os novos documentos em características tfidf
nd_tfidf = tfidf_trans.transform(new_doc_array)
nd_caracteristicas = np.round(nd_tfidf.todense(), 2)
display_caracteristicas(nd_caracteristicas, feature_names)

     00    01    02    03    05  1000    11    14    16    1h  ...  viagem  \
0  0.00  0.00  0.07  0.00  0.00  0.00  0.00  0.00  0.07  0.00  ...    0.00   
1  0.00  0.09  0.00  0.04  0.03  0.00  0.00  0.00  0.03  0.00  ...    0.00   
2  0.00  0.00  0.00  0.00  0.03  0.00  0.00  0.00  0.00  0.00  ...    0.04   
3  0.14  0.00  0.04  0.00  0.00  0.14  0.05  0.05  0.00  0.05  ...    0.00   

   viajar  vinculado  vocês  voltado  área  ótimo  última  única  úteis  
0    0.00       0.00   0.00     0.00  0.00   0.00    0.00   0.00   0.00  
1    0.03       0.00   0.00     0.00  0.04   0.04    0.00   0.00   0.07  
2    0.03       0.00   0.17     0.04  0.00   0.00    0.04   0.04   0.03  
3    0.00       0.05   0.00     0.00  0.00   0.00    0.00   0.00   0.00  

[4 rows x 410 columns]
    00    01   02   03   05  1000   11   14   16   1h  ...  viagem  viajar  \
0  0.0  0.11  0.0  0.0  0.0   0.0  0.0  0.0  0.0  0.0  ...     0.0     0.0   

   vinculado  vocês  voltado  área  ótimo  última  única  

In [ ]:
# Trecho de codigo original apresentou o erro abaixo e foi modificado
# TypeError: np.matrix is not supported. Please convert to a numpy array with np.asarray.
# For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html

# feature_names = bow_vectorizer.get_feature_names_out()
#Para construir um tfidf
# Para construir o transformador do tfidf e mostrar o corpus das caracteristicas tfidf
#tfidf_trans, tdidf_caracteristicas = tfidf_transformer(bow_caracteristicas)
#caracteristicas = np.round(tdidf_caracteristicas.todense(), 2)
#display_caracteristicas(caracteristicas, feature_names)
#nd_tfidf = tfidf_trans.transform(new_doc_caracteristicas)
#nd_caracteristicas = np.round(nd_tfidf.todense(), 2)
#display_caracteristicas(nd_caracteristicas, feature_names)
#feature_names = bow_vectorizer.get_feature_names_out()

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Inicializa vetorizador TF-IDF
tfidf_vectorizer = TfidfVectorizer()

# Converte bow_caracteristicas em uma lista de strings
documentos = [" ".join(map(str, documento)) for documento in bow_caracteristicas]

# Constroi um transformador TF-IDF
tfidf_trans = tfidf_vectorizer.fit_transform(documentos)

# Obtém nomes das características
feature_names = tfidf_vectorizer.get_feature_names_out()

# Extrai características TF-IDF como uma matriz densa
caracteristicas = np.round(tfidf_trans.toarray(), 2)

# Exibe características TF-IDF
display_caracteristicas(caracteristicas, feature_names)

# Converte novos documentos em uma lista de strings
novos_documentos_str = [" ".join(map(str, documento)) for documento in new_doc_caracteristicas]

# Transforma novos documentos em características TF-IDF
nd_tfidf = tfidf_vectorizer.transform(novos_documentos_str)

# Extrai características TF-IDF dos novos documentos como uma matriz densa
nd_caracteristicas = np.round(nd_tfidf.toarray(), 2)

# Exibr características TF-IDF dos novos documentos
display_caracteristicas(nd_caracteristicas, feature_names)

     10   100   101   107   108    11   111   118    12   120  ...    74  \
0  0.00  0.16  0.00  0.08  0.16  0.00  0.08  0.00  0.00  0.16  ...  0.00   
1  0.13  0.00  0.00  0.08  0.00  0.00  0.08  0.00  0.16  0.00  ...  0.00   
2  0.00  0.00  0.16  0.08  0.00  0.13  0.08  0.16  0.00  0.00  ...  0.16   
3  0.13  0.00  0.00  0.09  0.00  0.13  0.09  0.00  0.00  0.00  ...  0.00   

     81    82    85    87    88    91    92    96    98  
0  0.00  0.00  0.00  0.10  0.00  0.00  0.00  0.00  0.00  
1  0.13  0.00  0.16  0.00  0.16  0.00  0.00  0.00  0.00  
2  0.13  0.00  0.00  0.10  0.00  0.16  0.16  0.00  0.16  
3  0.00  0.17  0.00  0.11  0.00  0.00  0.00  0.17  0.00  

[4 rows x 146 columns]
    10  100  101  107  108   11  111  118   12  120  ...   74   81   82   85  \
0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0   

    87   88   91   92   96   98  
0  0.0  0.0  0.0  0.0  0.0  0.0  

[1 rows x 146 columns]
